In [1]:
!docker compose -f vertica.yml down --remove-orphans -v && docker compose -f vertica.yml up -d

[+] Running 0/0
 ⠋ Network storage_research_default  Creating                              0.1s 
[+] Running 1/1
 ✔ Network storage_research_default  Created                               0.1s 
 ⠿ Container vertica_ugc             Starting                              0.1s 
[+] Running 1/2
 ✔ Network storage_research_default  Created                               0.1s 
 ⠿ Container vertica_ugc             Starting                              0.2s 
[+] Running 1/2
 ✔ Network storage_research_default  Created                               0.1s 
 ⠿ Container vertica_ugc             Starting                              0.3s 
[+] Running 1/2
 ✔ Network storage_research_default  Created                               0.1s 
 ⠿ Container vertica_ugc             Starting                              0.4s 
[+] Running 1/2
 ✔ Network storage_research_default  Created                               0.1s 
 ⠿ Container vertica_ugc             Starting                              0.5s 
[+] Running 2

In [2]:
import vertica_python

connection_info = {
    'host': '127.0.0.1',
    'port': 5433,
    'user': 'dbadmin',
    'password': '',
    'autocommit': True,
}

conn = vertica_python.connect(**connection_info)

In [3]:
from typing import Iterable
from more_itertools import chunked


def create_table():
    sql = """
    CREATE TABLE IF NOT EXISTS view_history
    (   
        id IDENTITY,
        created_at DateTime,
        user_id VARCHAR(256) NOT NULL,
        film_id VARCHAR(256) NOT NULL,
        timestamp INTEGER NOT NULL
    )
    """
    cursor = conn.cursor()
    cursor.execute(sql)
    cursor.close()


def insert_data(data: Iterable[dict], chunk: int = 1000):
    sql = """
    INSERT INTO view_history (created_at, user_id, film_id, timestamp)
    VALUES (:created_at, :user_id, :film_id, :timestamp)
    """
    cursor = conn.cursor()
    for items in chunked(data, chunk):
        cursor.executemany(sql, items, use_prepared_statements=False)
    cursor.close()


def select_data():
    sql = """
    SELECT user_id, avg(timestamp) FROM view_history
    GROUP BY user_id
    """
    cursor = conn.cursor()
    cursor.execute(sql)
    cursor.close()


def clear_table():
    sql = """
    TRUNCATE TABLE view_history
    """
    cursor = conn.cursor()
    cursor.execute(sql)
    cursor.close()


In [4]:
import time
from collections import defaultdict


def bench(data: list[dict], chunk: int, n: int):
    elapsed = defaultdict(list)
    for _ in range(n):
        clear_table()
        start_time = time.time()
        insert_data(data, chunk)
        end_time = time.time()
        elapsed['write'].append(end_time - start_time)

        start_time = time.time()
        select_data()
        end_time = time.time()
        elapsed['read'].append(end_time - start_time)

    return elapsed


In [5]:
create_table()

In [6]:
from data import ViewHistoryCollection

total = 10_000_000
views = ViewHistoryCollection(total, int(total / 2), int(total / 4)).to_dict()

In [7]:
totals = [1_000, 10_000, 100_000, 1_000_000, 10_000_000]
elapsed = dict()
for total in totals:
    elapsed[total] = bench(views[: total + 1], 500, 3)

In [8]:
elapsed

{1000: defaultdict(list,
             {'write': [0.1579301357269287,
               0.09983015060424805,
               0.09468340873718262],
              'read': [0.006709575653076172,
               0.005630016326904297,
               0.0068438053131103516]}),
 10000: defaultdict(list,
             {'write': [0.8786861896514893,
               1.0265452861785889,
               0.8959033489227295],
              'read': [0.013215780258178711,
               0.012932538986206055,
               0.013453483581542969]}),
 100000: defaultdict(list,
             {'write': [10.200157642364502,
               9.935396194458008,
               9.683753728866577],
              'read': [0.036577701568603516,
               0.03526139259338379,
               0.04124093055725098]}),
 1000000: defaultdict(list,
             {'write': [94.44619178771973,
               93.93024849891663,
               95.68465566635132],
              'read': [0.31182217597961426,
               0.30826830863

In [9]:
import json

with open('result/vertica.json', 'w') as f:
    json.dump(elapsed, f)


In [10]:
conn.close()

In [11]:
!docker compose -f clickhouse_basic.yml down --remove-orphans -v

[+] Running 0/0
 ⠋ Container vertica_ugc  Stopping                                         0.1s 
[+] Running 0/1
 ⠙ Container vertica_ugc  Stopping                                         0.2s 
[+] Running 0/1
 ⠹ Container vertica_ugc  Stopping                                         0.3s 
[+] Running 0/1
 ⠸ Container vertica_ugc  Stopping                                         0.4s 
[+] Running 0/1
 ⠼ Container vertica_ugc  Stopping                                         0.5s 
[+] Running 0/1
 ⠴ Container vertica_ugc  Stopping                                         0.6s 
[+] Running 0/1
 ⠦ Container vertica_ugc  Stopping                                         0.7s 
[+] Running 0/1
 ⠧ Container vertica_ugc  Stopping                                         0.8s 
[+] Running 0/1
 ⠇ Container vertica_ugc  Stopping                                         0.9s 
[+] Running 0/1
 ⠏ Container vertica_ugc  Stopping                                         1.0s 
[+] Running 0/1
 ⠋ Container v